In [82]:
import os
os.environ['USE_PYGEOS'] = '0'
import requests
import pandas as pd
import geopandas as gpd
import datetime
from shapely import wkt, wkb
import polyline
from shapely.geometry import LineString, Point
import boto3
import numpy as np
import random
import json

In [2]:
def random_points_within(poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds

    points = []

    while len(points) < num_points:
        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)

    return points

def generate_points_within_tm_boundary(tm_boundary, trimet_crs):
    '''
    generate 2 points and make sure they are > 1 mile apart
    '''
    
    #the length of the line connecting the two points is the 
    #distance between them (as crow flies)
    #convert crs if not already
    tm_boundary_proj = tm_boundary.to_crs(trimet_crs)
    dist_btw_points = 0
    while dist_btw_points < 1:
        points_list = random_points_within(tm_boundary_proj.unary_union, 2)
        dist_btw_points = LineString(points_list).length/5280

    df = pd.DataFrame()
    df['points'] = points_list
    df['points'] = df['points'].apply(Point)
    gdf_points = gpd.GeoDataFrame(df,crs=trimet_crs ,geometry='points')
    gdf_points_reproject = gdf_points.to_crs("EPSG:4326")

    return (gdf_points_reproject, dist_btw_points)

def call_planner(fromPlace, toPlace):
    ''' 
    fromPlace = "lat, lon"
    toPlace = "lat,lon"
    '''
    date = datetime.datetime.now().strftime("%Y-%m-%d")
    base_url = "https://maps.trimet.org/otp_mod/plan"
    time="12:00"
    mode="WALK,BUS,TRAM,RAIL,GONDOLA"
    maxWalkDistance=5280/2
    walkSpeed=1.34
    numItineraries=3
    r = requests.get(url=base_url, params={'fromPlace':fromPlace, 'toPlace':toPlace, 'date':date, 'time':time
                                    ,'mode':mode, 'maxWalkDistance':maxWalkDistance, 'walkSpeed':walkSpeed
                                    ,'numItineraries':numItineraries})
    assert(r.status_code==200)
    return r.json()

def decode_create_leg_line(encoded_linestring):
    '''google polyline encoded linestring'''
    reformatted_coords = []
    original_coords = polyline.decode(encoded_linestring)
    for coord in original_coords:
        reformatted_coords.append((coord[1],coord[0]))
    return LineString(reformatted_coords)

def get_itinerary_paths(planner_response):
    ''' 
    planner_response = json_response from TriMet trip planner
    '''
    itineraries_df = pd.DataFrame()
    for itin_idx, itinerary in enumerate(planner_response['plan']['itineraries']):
        totalTime = itinerary['duration']
        walkTime = itinerary['walkTime']
        transitTime = itinerary['transitTime']
        waitingTime = itinerary['waitingTime']
        walkDistance = itinerary['walkDistance']
        for leg_idx, leg in enumerate(itinerary['legs']):
            route_id = leg.get('routeId','WALK').split(":")[-1]
            mode = leg['mode']
            fromName = leg['from']['name']
            toName = leg['to']['name']
            fromStopCode = leg['from'].get('stopCode','')
            toStopCode = leg['to'].get('stopCode','')
            legGeometry = decode_create_leg_line(leg['legGeometry']['points'])
            leg_df = pd.DataFrame([[itin_idx, totalTime, walkTime, transitTime, waitingTime, walkDistance,
                                   leg_idx, route_id, mode, fromStopCode, fromName, toStopCode, toName,
                                   legGeometry]],
                                   columns=['itin_idx', 'totalTime', 'walkTime', 'transitTime', 'waitingTime', 'walkDistance',
                                   'leg_id', 'route_id', 'mode', 'fromStopCode', 'fromName', 'toStopCode', 'toName',
                                   'legGeometry'])
            itineraries_df = pd.concat([itineraries_df,leg_df])
    return itineraries_df

## basic plan generate random points within trimet service area
**Steps**
1. get TriMet service area
2. generate 2 points
3. check that points are not within 1 mile of each other (as the crow flies)

## advanced plan generate random point within 2 portland block groups
**Steps**
1. Download block groups
2. get TriMet service area
3. generate points within two separate block groups - advanced no adjacent?

In [3]:
tm_boundary = gpd.read_file("gis/export/tm_route_buffer_bounds.geojson")
tm_boundary.head(3)

,description,geometry
0,trimet_route_buffer_boundary,"POLYGON ((-122.73188 45.54927, -122.73217 45.5..."


In [4]:
trimet_crs = "EPSG:2913"

In [47]:
gdf_points, dist_btw_points = generate_points_within_tm_boundary(tm_boundary, trimet_crs)
gdf_points.explore()

In [48]:
gdf_points['points_str'] = gdf_points['points'].apply(lambda x: f"{round(x.y,6)}, {round(x.x,6)}")

In [49]:
fromplace, toplace = gdf_points['points_str'].to_numpy()

In [50]:
json_content = call_planner(fromplace, toplace)

In [51]:
json_content

{'requestParameters': {'date': '2023-05-05',
  'mode': 'WALK,BUS,TRAM,RAIL,GONDOLA',
  'walkSpeed': '1.34',
  'fromPlace': '45.429642, -122.721025',
  'toPlace': '45.525905, -122.969953',
  'time': '12:00',
  'maxWalkDistance': '2640.0',
  'numItineraries': '3'},
 'plan': {'date': 1683313200000,
  'from': {'name': 'Origin',
   'wheelchairBoarding': 'UNKNOWN',
   'lon': -122.721025,
   'lat': 45.429642,
   'orig': '',
   'vertexType': 'NORMAL'},
  'to': {'name': 'Destination',
   'wheelchairBoarding': 'UNKNOWN',
   'lon': -122.969953,
   'lat': 45.525905,
   'orig': '',
   'vertexType': 'NORMAL'},
  'itineraries': [{'duration': 6741,
    'startTime': 1683314480000,
    'endTime': 1683321221000,
    'walkTime': 1422,
    'transitTime': 4491,
    'waitingTime': 828,
    'walkDistance': 1635.0370298365717,
    'walkLimitExceeded': False,
    'elevationLost': 22.00000000000002,
    'elevationGained': 26.75000000000003,
    'transfers': 1,
    'fare': {'fare': {'regular': {'currency': {'symb

In [52]:
itineraries_df = get_itinerary_paths(json_content)

In [53]:
itineraries_df.head(2)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry
0,0,6741,1422,4491,828,1635.03703,0,WALK,WALK,,Origin,10305,Kerr & Touchstone,"LINESTRING (-122.72104 45.42963, -122.72115 45..."
0,0,6741,1422,4491,828,1635.03703,1,78,BUS,10305,Kerr & Touchstone,9985,Beaverton Transit Center,"LINESTRING (-122.71475 45.42806, -122.71479 45..."


In [54]:
itineraries_gdf = gpd.GeoDataFrame(itineraries_df, crs="4326", geometry="legGeometry")
itineraries_gdf.head(2)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry
0,0,6741,1422,4491,828,1635.03703,0,WALK,WALK,,Origin,10305,Kerr & Touchstone,"LINESTRING (-122.72104 45.42963, -122.72115 45..."
0,0,6741,1422,4491,828,1635.03703,1,78,BUS,10305,Kerr & Touchstone,9985,Beaverton Transit Center,"LINESTRING (-122.71475 45.42806, -122.71479 45..."


In [65]:
unique_combinations = itineraries_df[itineraries_df['route_id']!='WALK'].groupby('itin_idx').agg(route_id_list=('route_id',list)).reset_index().drop_duplicates(subset='route_id_list')
unique_combinations['route_id_combo'] = unique_combinations['route_id_list'].apply(lambda x: " to ".join(x))
unique_combinations

,itin_idx,route_id_list,route_id_combo
0,0,"[78, 100]",78 to 100
1,1,"[78, 57]",78 to 57


In [67]:
itineraries_reduced = itineraries_gdf.merge(unique_combinations, how='inner', on='itin_idx')
itineraries_reduced.head(3)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_list,route_id_combo
0,0,6741,1422,4491,828,1635.03703,0,WALK,WALK,,Origin,10305,Kerr & Touchstone,"LINESTRING (-122.72104 45.42963, -122.72115 45...","[78, 100]",78 to 100
1,0,6741,1422,4491,828,1635.03703,1,78,BUS,10305,Kerr & Touchstone,9985,Beaverton Transit Center,"LINESTRING (-122.71475 45.42806, -122.71479 45...","[78, 100]",78 to 100
2,0,6741,1422,4491,828,1635.03703,2,WALK,WALK,9985,Beaverton Transit Center,9818,Beaverton TC MAX Station,"LINESTRING (-122.80130 45.49138, -122.80132 45...","[78, 100]",78 to 100


In [68]:
m = itineraries_reduced[itineraries_reduced['itin_idx']==0].explore(column='route_id', tiles='CartoDB positron')
gdf_points.explore(m=m, color='red')

In [118]:
import folium
import seaborn as sns

In [57]:
itineraries_centroid = itineraries_reduced.unary_union.convex_hull.centroid

In [133]:
# Get Unique continents
color_labels = itineraries_reduced['route_id'].unique()

# List of colors in the color palettes
hex_values = sns.color_palette("Set2", len(color_labels)).as_hex()

# Map continents to the colors
color_map = dict(zip(color_labels, hex_values))

In [134]:
color_map['WALK']

'#66c2a5'

In [135]:
m = folium.Map(location=[itineraries_centroid.y, itineraries_centroid.x], zoom_start=13, tiles='CartoDB positron')
origin_dest_points = folium.GeoJson(gdf_points.to_json(), name='origin_destination')
origin_dest_points.add_to(m)

for idx, itin_group in itineraries_reduced.groupby('itin_idx'):
    itin_idx = itin_group.iloc[0]['itin_idx']
    itin_route_combo = itin_group.iloc[0]['route_id_combo']
    group_json = itin_group.to_json()
    folium.GeoJson(group_json, name=f'{itin_route_combo}'
                   , tooltip=folium.GeoJsonTooltip(fields=['route_id'])
                   ,style_function=lambda feature: {
                                    'color': color_map[feature['properties']['route_id']],
                                    'weight': 1
                                }
                   ).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m

In [130]:
group_json

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"itin_idx": 0, "totalTime": 6741, "walkTime": 1422, "transitTime": 4491, "waitingTime": 828, "walkDistance": 1635.0370298365717, "leg_id": 0, "route_id": "WALK", "mode": "WALK", "fromStopCode": "", "fromName": "Origin", "toStopCode": "10305", "toName": "Kerr & Touchstone", "route_id_list": ["78", "100"], "route_id_combo": "78 to 100"}, "geometry": {"type": "LineString", "coordinates": [[-122.72104, 45.42963], [-122.72115, 45.4297], [-122.72121, 45.42973], [-122.72116, 45.42978], [-122.72111, 45.42983], [-122.72105, 45.42987], [-122.72099, 45.42992], [-122.72092, 45.42996], [-122.72085, 45.43], [-122.72058, 45.42983], [-122.72053, 45.4298], [-122.72045, 45.42978], [-122.72037, 45.42976], [-122.72016, 45.42969], [-122.72009, 45.42968], [-122.72, 45.42964], [-122.71994, 45.42961], [-122.7199, 45.42958], [-122.7198, 45.42945], [-122.71973, 45.42939], [-122.71966, 45.42936], [-122.71965, 45.42935], [-1

In [ ]:
folium.GeoJsonTooltip(fields='route_id')

In [90]:
m = folium.Map(location=[itineraries_centroid.y, itineraries_centroid.x], zoom_start=12, tiles='CartoDB positron')

itinerary_json = itineraries_reduced[itineraries_reduced['itin_idx']==0].to_json()

folium.GeoJson(itinerary_json, name=f'{itin_route_combo}', tooltip=folium.GeoJsonTooltip(fields=['route_id'])).add_to(m)

m

In [88]:
json.loads(itinerary_json)['features'][0]['properties']['route_id']

'WALK'

In [ ]:
itineraries_gdf.to_file("gis/export/itineraries.geojson", driver="GeoJSON")

In [ ]:
client = boto3.client("s3")
client.upload_file("gis/export/itineraries.geojson", "meysohn-sandbox", "trimet_trip_planner/itineraries.geojson",ExtraArgs={'ACL':'public-read'})

In [ ]:
# fromplace = "45.504286,-122.646199"
# toplace = "45.542102,-122.664715"

to_from_df = 